# Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, sys
from pathlib import Path
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break

from data_wrangling import specific_utils, process_monkey_information, general_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features
from visualization.matplotlib_tools import plot_behaviors_utils
from neural_data_analysis.neural_analysis_tools.get_neural_data import neural_data_processing
from neural_data_analysis.neural_analysis_tools.visualize_neural_data import plot_neural_data, plot_modeling_result
from neural_data_analysis.neural_analysis_tools.model_neural_data import transform_vars, neural_data_modeling, drop_high_corr_vars, drop_high_vif_vars
from neural_data_analysis.topic_based_neural_analysis.neural_vs_behavioral import prep_monkey_data, prep_target_data, neural_vs_behavioral_class
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils, pn_helper_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class, cca_utils, cca_cv_utils
from neural_data_analysis.neural_analysis_tools.cca_methods.cca_plotting import cca_plotting, cca_plot_lag_vs_no_lag, cca_plot_cv
from machine_learning.ml_methods import regression_utils, ml_methods_utils, regz_regression_utils, ml_methods_class, classification_utils, ml_plotting_utils
from null_behaviors import find_best_arc, curvature_utils, curv_of_traj_utils, opt_arc_utils

import sys
import math
import gc
import subprocess
from pathlib import Path
from importlib import reload

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import linalg, interpolate
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy import sparse
import torch
from numpy import pi

# Machine Learning imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.cancorr import CanCorr

# Neuroscience specific imports
import neo
import rcca

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
print("done")

%load_ext autoreload
%autoreload 2

# Retrieve data

## get data

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0328"

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0402"

In [ ]:
reload(opt_arc_utils)

In [ ]:
reduce_y_var_lags = True
planning_data_by_point_exists_ok = True
y_data_exists_ok = True

pn = planning_and_neural_class.PlanningAndNeural(raw_data_folder_path=raw_data_folder_path)
pn.prep_data_to_analyze_planning(planning_data_by_point_exists_ok=planning_data_by_point_exists_ok)
pn.planning_data_by_point, cols_to_drop = general_utils.drop_columns_with_many_nans(
    pn.planning_data_by_point)
pn.get_x_and_y_data_for_modeling(exists_ok=y_data_exists_ok, reduce_y_var_lags=reduce_y_var_lags)

In [ ]:
stop!

## compare dist of every var

In [ ]:
test_data = pn.y_var[pn.y_var['whether_test'] == 1]
ctr_data = pn.y_var[pn.y_var['whether_test'] == 0]

for col in test_data.columns:
#for col in ['target_index']:
    # compare the distribution through histplot (by percentage) of the column in test_data and ctr_data
    plt.figure(figsize=(6, 4))
    sns.histplot(test_data[col].values, label='test', alpha=0.5, stat='percent', kde=True, bins=100)
    sns.histplot(ctr_data[col].values, label='ctr', alpha=0.5, stat='percent', kde=True, bins=100)
    plt.title(f'{col}', fontsize=14)
    plt.xlabel(f'{col}', fontsize=12)
    plt.ylabel('percentage', fontsize=12)
    plt.legend()
    plt.show()
    

## check NA

In [ ]:
general_utils.check_na_in_df(pn.planning_data_by_point)

## var's corr

In [ ]:
nxt_cols = [col for col in pn.y_var.columns if 'nxt' in col]
corr_df = pn.y_var.corr()[nxt_cols]

In [ ]:
ml_plotting_utils.plot_correlation_heatmap(corr_df)

# LR: on all data together

### Just nxt ff vars

In [ ]:
columns_of_interest = ['whether_test']

In [ ]:
columns_of_interest = [col for col in pn.y_var.columns if 'nxt' in col]

In [ ]:
x_var, y_var = pn.get_x_and_y_var_for_lr(test_or_control='test', use_x_var_lags=True)

y_var_sub = y_var[columns_of_interest]
# With x var lags
y_var_lr_df = neural_data_modeling.get_y_var_lr_df(
    x_var, y_var_sub, verbose=True)

## plot

In [ ]:
# Plot features from y_var_lr_df
max_plot_number = 10
count = 0
bins_to_plot = range(len(y_var))

for i, column in enumerate(y_var_lr_df.feature.values): # so that features are plotted in the order of correlation
    if i >= max_plot_number:
        break
    plot_neural_data.plot_regression(y_var, column, x_var, bins_to_plot=None, min_r_squared_to_plot=0.3)

# LR: train-test split

## regularized

In [ ]:
columns_of_interest = ['whether_test']

In [ ]:
reload(regz_regression_utils)

In [ ]:
x_var, y_var = pn.get_x_and_y_var_for_lr(test_or_control='both', use_x_var_lags=True)
X_train, X_test, y_train, y_test = ml_methods_utils.train_test_split_based_on_segments(x_var, y_var)

# Basic usage with comprehensive metrics
for y_var_column in columns_of_interest:
    print('y_var_column:', y_var_column)
    y_train_var =  y_train[y_var_column]   
    y_test_var = y_test[y_var_column]

    results, results_df, y_pred_train, y_pred_test = regz_regression_utils.regularized_regression(
        X_train, y_train_var, X_test, y_test_var, method='ridge', alpha=1.0
    )
    print(results_df)

    # # Create comprehensive report
    report = regz_regression_utils.regression_metrics_report(
        y_test_var, y_pred_test, model_name="Ridge Regression", show_plots=True
    )
        
    results, results_df = regz_regression_utils.compare_regularized_models(X_train, y_train_var, X_test, y_test_var, verbose=True, show_plots=False)
    regz_regression_utils.print_model_comparison_summary(results)
    

## split based on targets

In [ ]:
# select 'random_dummy'
pn.y_var = pn_utils.randomly_assign_random_dummy_based_on_targets(pn.y_var)
columns_of_interest = ['random_dummy']

In [ ]:
columns_of_interest = ['whether_test']

In [ ]:
# select all nxt_ff variables
columns_of_interest = [col for col in pn.y_var.columns if 'nxt' in col]

In [ ]:
# select all cur_ff variables
columns_of_interest = [col for col in pn.y_var.columns if 'cur' in col]

In [ ]:
x_var, y_var = pn.get_x_and_y_var_for_lr(test_or_control='both', use_x_var_lags=True)
ml_methods_utils.run_segment_split_regression(x_var, y_var, columns_of_interest, segment_column='segment')

## random split

In [ ]:
x_var, y_var = pn.get_x_and_y_var_for_lr(test_or_control='both', use_x_var_lags=True)

columns_of_interest = [col for col in pn.y_var.columns if 'nxt' in col]

ml_inst = ml_methods_class.MlMethods()
for y_var_column in columns_of_interest:
    print('y_var_column:', y_var_column)
    ml_inst.split_and_use_linear_regression(pn.x_var_lags_reduced, pn.y_var[[y_var_column]])


# CCA

https://medium.com/@pozdrawiamzuzanna/canonical-correlation-analysis-simple-explanation-and-python-example-a5b8e97648d2

## check data

In [ ]:
X1 = pn.x_var_reduced[[col for col in pn.x_var_reduced.columns if col.startswith('cluster_')]].drop(columns=['bin', 'segment'], errors='ignore')
X2 = pn.y_var_reduced.copy()
#X2 = pn.y_var_reduced.drop(columns=['bin', 'segment'], errors='ignore')

X1_lags = pn.x_var_lags_reduced[[col for col in pn.x_var_lags_reduced.columns if col.startswith('cluster_')]].drop(columns=['bin', 'segment'], errors='ignore')
X2_lags = pn.y_var_lags_reduced.copy()
# X2_lags = pn.y_var_lags_reduced.drop(columns=['bin', 'segment'], errors='ignore')

In [ ]:
vif_df = drop_high_vif_vars.get_vif_df(X1)
print('X1 vif_df: ', vif_df.head(7))

vif_df = drop_high_vif_vars.get_vif_df(X2)
print('X2 vif_df: ', vif_df.head(7))

vif_df = drop_high_vif_vars.get_vif_df(X1_lags)
print('X1_lags vif_df: ', vif_df.head(7))

vif_df = drop_high_vif_vars.get_vif_df(X2_lags)
print('X2_lags vif_df: ', vif_df.head(7))

## conduct cca

In [ ]:
# If error occurs, try reducing y_var again, and can also try pn.y_var_reduced.corr()[pn.y_var_reduced.corr() > 0.9]


X1 = pn.x_var_reduced[[col for col in pn.x_var_reduced.columns if col.startswith('cluster_')]].drop(columns=['bin', 'segment'], errors='ignore').copy()
X2 = pn.y_var_reduced.copy() #drop(columns=['bin', 'segment'], errors='ignore').copy()
cca_no_lag = cca_class.CCAclass(X1=X1, 
                                X2=X2, 
                                lagging_included=False)
cca_no_lag.conduct_cca()

X1_lags = pn.x_var_lags_reduced[[col for col in pn.x_var_lags_reduced.columns if col.startswith('cluster_')]].drop(columns=['bin', 'segment'], errors='ignore').copy()
X2_lags = pn.y_var_lags_reduced.drop(columns=['bin', 'segment'], errors='ignore').copy()
cca_lags = cca_class.CCAclass(X1=X1_lags, 
                              X2=X2_lags, 
                              lagging_included=True)
# for all columns that end with _0, rename them to the column name without the _0
cca_lags.X2.columns = cca_lags.X2.columns.str.replace('_0', '')
cca_lags.conduct_cca()


print(f'pn.x_var_lags.shape: {pn.x_var_lags.shape}')
print(f'pn.y_var_lags_reduced.shape: {pn.y_var_lags_reduced.shape}')

cca_inst = cca_lags

## compare lag vs no lag

In [ ]:
can_load_df = pd.DataFrame(cca_no_lag.canon_corr, columns = ['no_lag'])
can_load_df[f'with_lags'] = cca_lags.canon_corr
can_load_df['component'] = [f'CC {i+1}' for i in range(cca_lags.n_components)]
# convert can_load_df to long format
can_load_df_long = pd.melt(can_load_df, id_vars=['component'], var_name='lag', value_name='canon_coeff')

In [ ]:
# make a sns bar plot on can_load_df_long
plt.figure(figsize=(8, 6))
sns.barplot(x='component', y='canon_coeff', data=can_load_df_long, hue='lag')
plt.show()

## cca_inst (choose one between lags and no lag)

In [ ]:
# choose no lag
cca_inst = cca_no_lag

In [ ]:
# choose lags
cca_inst = cca_lags

## test for p values

In [ ]:
cca_inst.test_for_p_values()

# Heatmap of loadings

## X1 loadings

In [ ]:
cca_no_lag.plot_X1_loadings()

## X2 loadings

In [ ]:
cca_no_lag.plot_X2_loadings()

In [ ]:
cca_lags.plot_X2_loadings()

# Transform vars (e.g. use basis functions)

In [ ]:
cca_no_lag.X2_tf_df = transform_vars.transform_behav_data(cca_no_lag.X2)
cca_lags.X2_tf_df = transform_vars.transform_behav_data(cca_lags.X2)

In [ ]:
# If need to use the data
cca_inst = cca_lags
X1_df = cca_inst.X1_sc_df
X2_df = cca_inst.X2_tf_df

# Lags vs no lag & train vs test

## grouped by segment

In [ ]:
combined_can_load_df

In [ ]:
combined_can_load_df

In [ ]:
combined_cross_view_df, combined_can_load_df = cca_cv_utils.combine_cca_cv_results(cca_no_lag, cca_lags, n_components=7, reg=0.1, n_splits=7,
                                                                               groups=pn.y_var['segment'].values
                                                                               )

### cross-view X1

In [ ]:
# dataset_name = 'X1'
# cross_view_sub = combined_cross_view_df[combined_cross_view_df['dataset'] == dataset_name]
# cca_plot_lag_vs_no_lag.plot_cca_lag_vs_nolag_and_train_vs_test(cross_view_sub, dataset_name, mode='lag_offset')


### cross-view X2

In [ ]:
pn.y_var['segment_duration'].describe()

In [ ]:
features = ['cur_ff_distance', 'cur_ff_angle', 'cur_ff_rel_x', 'cur_ff_rel_y', 'abs_cur_ff_angle', 'abs_cur_ff_rel_x',
            'nxt_ff_distance', 'nxt_ff_angle', 'nxt_ff_rel_x', 'nxt_ff_rel_y', 'abs_nxt_ff_angle', 'abs_nxt_ff_rel_x',
            'time_rel_to_stop', 'time_since_last_capture', 'whether_test']
aug_features = []
for feat in features:
    aug_features.extend([var for var in combined_cross_view_df.variable.values if feat in var])


In [ ]:
dataset_name = 'X2'
cross_view_sub = combined_cross_view_df[combined_cross_view_df['dataset'] == dataset_name]
cross_view_sub = cross_view_sub[cross_view_sub['variable'].isin(aug_features)]
cca_plot_lag_vs_no_lag.plot_cca_lag_vs_nolag_and_train_vs_test(cross_view_sub, dataset_name, mode='lag_offset',
                                                                title_prefix='Split by segment: ')


### all 'significant' features

In [ ]:
dataset_name = 'X2'
cross_view_sub = combined_cross_view_df[combined_cross_view_df['dataset'] == dataset_name]
cross_view_sub= cross_view_sub[np.abs(cross_view_sub['mean_corr']) > 0.05]
cca_plot_lag_vs_no_lag.plot_cca_lag_vs_nolag_and_train_vs_test(cross_view_sub, dataset_name, mode='lag_offset',
                                                                title_prefix='Split by segment: ')


## not grouped by segment

In [ ]:
combined_cross_view_df, combined_can_load_df = cca_cv_utils.combine_cca_cv_results(cca_no_lag, cca_lags, n_components=7, reg=0.1, n_splits=7,
                                                                               #groups=pn.y_var['segment'].values
                                                                               )

### cross-view X1

In [ ]:
# dataset_name = 'X1'
# cross_view_sub = combined_cross_view_df[combined_cross_view_df['dataset'] == dataset_name]
# cca_plot_lag_vs_no_lag.plot_cca_lag_vs_nolag_and_train_vs_test(cross_view_sub, dataset_name, mode='lag_offset')


### cross-view X2

In [ ]:
features = ['cur_ff_distance', 'cur_ff_angle', 'cur_ff_rel_x', 'cur_ff_rel_y', 'abs_cur_ff_angle', 'abs_cur_ff_rel_x',
            'nxt_ff_distance', 'nxt_ff_angle', 'nxt_ff_rel_x', 'nxt_ff_rel_y', 'abs_nxt_ff_angle', 'abs_nxt_ff_rel_x',
            'time_rel_to_stop', 'time_since_last_capture', 'whether_test']
aug_features = []
for feat in features:
    aug_features.extend([var for var in combined_cross_view_df.variable.values if feat in var])


In [ ]:
dataset_name = 'X2'
cross_view_sub = combined_cross_view_df[combined_cross_view_df['dataset'] == dataset_name]
cross_view_sub = cross_view_sub[cross_view_sub['variable'].isin(aug_features)]
cca_plot_lag_vs_no_lag.plot_cca_lag_vs_nolag_and_train_vs_test(cross_view_sub, dataset_name, mode='lag_offset',
                                                               title_prefix='Not split by segment: ')


In [ ]:
stop!

# Just train vs test

In [ ]:
features = ['cur_ff_distance', 'cur_ff_angle', 'cur_ff_rel_x', 'cur_ff_rel_y', 'abs_cur_ff_angle', 'abs_cur_ff_rel_x',
            'nxt_ff_distance', 'nxt_ff_angle', 'nxt_ff_rel_x', 'nxt_ff_rel_y', 'abs_nxt_ff_angle', 'abs_nxt_ff_rel_x',
            'time_rel_to_stop', 'time_since_last_capture', 'whether_test']
aug_features = []
for feat in features:
    aug_features.extend([var for var in combined_cross_view_df.variable.values if feat in var])

## cross-view X1

In [ ]:
# filter_significant = True
# sort_by_significance = True
# significance_threshold = 4
# whether_lag = 'lag'

# combined_cross_view_df_sub = combined_cross_view_df[combined_cross_view_df['whether_lag'] == whether_lag]

# # X1
# cca_plot_cv.plot_cca_cv_results(combined_cross_view_df_sub, data_type='X1',
#                                     filter_significant=filter_significant, sort_by_significance=sort_by_significance, significance_threshold=significance_threshold)


## cross-view X2

### lag

In [ ]:
filter_significant = True
sort_by_significance = True
significance_threshold = 1
whether_lag = 'lag'

combined_cross_view_df_sub = combined_cross_view_df[combined_cross_view_df['whether_lag'] == whether_lag]

# X2
cca_plot_cv.plot_cca_cv_results(combined_cross_view_df_sub, data_type='X2',
                                    filter_significant=filter_significant, sort_by_significance=sort_by_significance, significance_threshold=significance_threshold)


### no lag

In [ ]:
combined_cross_view_df_sub['trial']

In [ ]:
filter_significant = True
sort_by_significance = True
significance_threshold = 1
whether_lag = 'no_lag'

combined_cross_view_df_sub = combined_cross_view_df[combined_cross_view_df['whether_lag'] == whether_lag]

# X2
cca_plot_cv.plot_cca_cv_results(combined_cross_view_df_sub, data_type='X1',
                                    filter_significant=filter_significant, sort_by_significance=sort_by_significance, significance_threshold=significance_threshold)


In [ ]:
filter_significant = True
sort_by_significance = True
significance_threshold = 1
whether_lag = 'no_lag'

combined_cross_view_df_sub = combined_cross_view_df[combined_cross_view_df['whether_lag'] == whether_lag]

# X2
cca_plot_cv.plot_cca_cv_results(combined_cross_view_df_sub, data_type='X2',
                                    filter_significant=filter_significant, sort_by_significance=sort_by_significance, significance_threshold=significance_threshold)


# CCA exp

In [ ]:
from neural_data_analysis.neural_analysis_tools.cca_methods.advanced_cca_tools import cca_temporal_tools
reload(cca_temporal_tools)

In [ ]:
from neural_data_analysis.neural_analysis_tools.cca_methods.advanced_cca_tools.cca_temporal_tools import KCCA

# train test split
X_train, X_test, Y_train, Y_test = train_test_split(X1.values, X2.values, test_size=0.3, random_state=42)

kcca = KCCA(n_components=2, kernel_x='rbf', kernel_y='rbf',
            reg_x=1e-2, reg_y=1e-2, gamma_x=0.5, gamma_y=0.5)
kcca.fit(X_train, Y_train)

U_train, V_train = kcca.transform(X_train, Y_train)
U_test, V_test   = kcca.transform(X_test, Y_test)

import numpy as np
from neural_data_analysis.neural_analysis_tools.cca_methods.advanced_cca_tools.cca_temporal_tools import RCCA, KCCA, spectral_cca, raised_cosine_basis, embed_with_basis

# X: (T, p) neural; Y: (T, q) behavior
X = X1.values 
Y = X2.values

# 1) Ridge CCA (linear)
rcca = RCCA(n_components=3, reg_x=1e-2, reg_y=1e-2, scale=False).fit(X, Y)
U, V = rcca.transform(X, Y)  # canonical variates
print("corrs:", rcca.corrs_)

# 2) Kernel CCA (RBF)
kcca = KCCA(n_components=3, kernel_x='rbf', kernel_y='rbf', reg_x=1e-2, reg_y=1e-2).fit(X, Y)
Uk, Vk = kcca.transform(X, Y)
print("kernel corrs:", kcca.corrs_)

# 3) Spectral CCA (canonical coherence spectrum)
spec = spectral_cca(X, Y, fs=1.0, nperseg=512, noverlap=256, reg_x=1e-3, reg_y=1e-3, n_components=1)
f_peak = spec.freqs[spec.coh[0].argmax()]
print("Peak coherence freq:", f_peak, "value:", spec.coh[0].max())

# 4) Reduced-basis embedding + RCCA
lags, B = raised_cosine_basis(n_basis=6, t_max=10, dt=1.0, log_spaced=True)
Xb, Yb = embed_with_basis(X, B), embed_with_basis(Y, B)
rcca_b = RCCA(n_components=3, reg_x=1e-2, reg_y=1e-2).fit(Xb, Yb)


# Appendix

## find cutoff based on curv_of_traj

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0328"
pn = planning_and_neural_class.PlanningAndNeural(raw_data_folder_path=raw_data_folder_path)

pn.retrieve_or_make_monkey_data()
pn.make_or_retrieve_ff_dataframe()
# pn.ff_dataframe = pn.ff_dataframe.iloc[:1000].copy()

In [ ]:
pn.make_curvature_df(clean=False)

In [ ]:
reload(opt_arc_utils)

In [ ]:
col = 'curv_of_traj'
n = 5
arr = pn.curvature_df[col].copy()
opt_arc_utils.plot_winsorization_with_cutoffs(arr, method="std", n=n)
mean, std = arr.mean(), arr.std()
lower, upper = mean - n * std, mean + n * std
print('min: ', np.min(arr), 'max: ', np.max(arr))
print('lower: ', lower, 'upper: ', upper)

In [ ]:
n = 5
mean, std = arr.mean(), arr.std()
lower, upper = mean - n * std, mean + n * std
print('min: ', np.min(arr), 'max: ', np.max(arr))
print('lower: ', lower, 'upper: ', upper)

## check winsorization on other columns

In [ ]:


for col in ['curv_of_traj', 'curvature_lower_bound', 'curvature_upper_bound', 'opt_arc_curv', 'curv_diff', 'abs_curv_diff']:
    data = pn.curvature_df[col].copy()
    
    print('========================= col: ', col, '=========================')
    opt_arc_utils.plot_winsorization_with_cutoffs(data, method="std", n=4)
    data = opt_arc_utils.winsorize_func(data)
    print('After chained winsorization')
    opt_arc_utils.plot_winsorization_with_cutoffs(data, method="std", n=4)
    #plot_winsorization_with_cutoffs(data, method="percentile", lower_pct=1, upper_pct=99)

## reduce y_var only by vif

In [ ]:
stop!

In [ ]:
pn.reduce_y_var(save_data=True,
                     corr_threshold_for_lags_of_a_feature=0.97,
                     vif_threshold_for_initial_subset=5, vif_threshold=5, verbose=True,
                     filter_corr_by_all_columns=False,
                     filter_vif_by_subsets=False,
                     filter_vif_by_all_columns=True,
                     exists_ok=False,
                     )

## check final VIF

In [ ]:
vif_df = drop_high_vif_vars.get_vif_df(pn.y_var_reduced)
vif_df

In [ ]:
vif_df = drop_high_vif_vars.get_vif_df(pn.y_var_lags_reduced)
vif_df

## check correlations in y_var_lags

In [ ]:
# sort pn.y_var_lags by column str a to z
pn.y_var_lags2 = pn.y_var_lags.reindex(sorted(pn.y_var_lags.columns), axis=1)

# # sort pn.y_var_lags by column str z to a
# pn.y_var_lags_reduced = pn.y_var_lags_reduced.reindex(sorted(pn.y_var_lags_reduced.columns, reverse=True), axis=1)

In [ ]:
pn.y_var_lags2.iloc[:, :10]

In [ ]:
pn.y_var_lags2.iloc[:, :10].corr()

## check high corr within feature's lagged columns

In [ ]:
df_with_lags = pn.y_var_lags2.copy()
num_original_columns = len(df_with_lags.columns)
base_features = drop_high_corr_vars.get_base_feature_names(df_with_lags)
columns_dropped = []
top_values_by_feature = pd.DataFrame()
for i, feature in enumerate(base_features):
    df_with_lags_sub = drop_high_corr_vars._find_subset_of_df_with_lags_for_current_feature(
        df_with_lags, feature)
    # temp_columns_to_drop, top_values_of_feature = drop_high_corr_vars._drop_lags_for_feature(
    #     df_with_lags, feature, corr_threshold, vif_threshold, use_vif_instead_of_corr, drop_lag_0_last_in_vif)
    if df_with_lags_sub.corr().iloc[1,2] == 1:
        print(feature)

## Compare columns in behav_data (target_decoder) and final_behavioral_data (neural_vs_behavioral_class)

In [ ]:
pn.get_behav_data(exists_ok=False)

In [ ]:
data_item = neural_vs_behavioral_class.NeuralVsBehavioralClass(raw_data_folder_path=raw_data_folder_path)
data_item.streamline_preparing_neural_and_behavioral_data()

In [ ]:
# Columns in pn.behav_data_by_bin but not in data_item.final_behavioral_data
only_in_pn = set(pn.behav_data_by_bin.columns) - set(data_item.final_behavioral_data.columns)
print("Columns only in pn.behav_data_by_bin:")
only_in_pn = np.array(sorted(only_in_pn))
print(only_in_pn)
print('\n \n')

# Columns in data_item.final_behavioral_data but not in pn.behav_data_by_bin
final_behavioral_data_columns = data_item.final_behavioral_data.columns
# remove all 'avg_bin_' prefix
final_behavioral_data_columns = [col.replace('avg_bin_', '') for col in final_behavioral_data_columns]
only_in_data_item = set(final_behavioral_data_columns) - set(pn.behav_data_by_bin.columns)
print("Columns only in data_item.final_behavioral_data:")
only_in_data_item = np.array(sorted(only_in_data_item))
print(only_in_data_item)

In [ ]:
pn.cur_and_nxt_ff_from_ref_df.columns

In [ ]:
pn.test_plan_data_inst.df.columns

## just get planning_timestep_data

In [ ]:
planning_data_by_point_exists_ok = False

raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330"
ref_point_mode='time after cur ff visible'
ref_point_value=0.1
normalize = False
eliminate_outliers = False
use_curv_to_ff_center = False
curv_of_traj_mode = 'distance'
window_for_curv_of_traj=[-25, 0]
truncate_curv_of_traj_by_time_of_capture = True

   # get behavioral_data
ph = pn_helper_class.PlanningAndNeuralHelper(raw_data_folder_path=raw_data_folder_path)

ph.load_raw_data(raw_data_folder_path)
ph.prep_behav_data_to_analyze_planning(ref_point_mode=ref_point_mode,
                                                            ref_point_value=ref_point_value,
                                                            curv_of_traj_mode=curv_of_traj_mode,
                                                            window_for_curv_of_traj=window_for_curv_of_traj,
                                                            truncate_curv_of_traj_by_time_of_capture=truncate_curv_of_traj_by_time_of_capture,
                                                            use_curv_to_ff_center=use_curv_to_ff_center,
                                                            eliminate_outliers=eliminate_outliers,
                                                            planning_data_by_point_exists_ok=planning_data_by_point_exists_ok
                                                            )

## check corr

In [ ]:
pn.y_var_reduced[['seg_start_time', 'seg_end_time']].corr()

In [ ]:
ml_plotting_utils.plot_correlation_heatmap(pn.y_var_reduced.corr(), annotation_threshold=0.3)

## exp

In [ ]:
for col in ['curv_of_traj', 'curvature_lower_bound', 'curvature_upper_bound', 'opt_arc_curv', 'curv_diff', 'abs_curv_diff']:
    data = pn.curvature_df[col]
    
    winsor_std = winsorize(data, method="std", n=3)
    winsor_pct = winsorize(data, method="percentile", lower_pct=1, upper_pct=99)

    print("Original min/max:", data.min(), data.max())
    print("3σ Winsorized min/max:", winsor_std.min(), winsor_std.max())
    print("1–99% Winsorized min/max:", winsor_pct.min(), winsor_pct.max())